#Classification Using Keras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

sns.set_style()
sns.set_theme()

drive.mount('/drive')

plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
df = pd.read_csv("/drive/My Drive/Colab Notebooks/Deep Learning Course (Jose Portilia)/4. Basic Artificial Neural Networks - ANNs/cancer_classification.csv")

##EDA and Preprocessing

###Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
sns.countplot(x = 'benign_0__mal_1', data = df)

In [ ]:
df.corr()['benign_0__mal_1'].sort_values()[:-1].plot(kind = 'bar')

In [ ]:
sns.heatmap(df.corr())

###Data Preprocessing

In [ ]:
X = df.drop('benign_0__mal_1', axis = 1).values
y = df['benign_0__mal_1'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##Dealing With Overfitting and Evaluation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()

model.add(Dense(30, activation = 'relu'))

model.add(Dense(15, activation = 'relu'))

#Binary Classification
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [ ]:
model.fit(X_train, y_train, epochs = 600, validation_data = (X_test, y_test))

In [ ]:
losses = pd.DataFrame(model.history.history)

losses.plot()

###Tackling Overfitting using EarlyStopping

In [ ]:
model = Sequential()

model.add(Dense(30, activation = 'relu'))

model.add(Dense(15, activation = 'relu'))

#Binary Classification
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 25)

In [ ]:
model.fit(X_train, y_train, epochs = 600, validation_data = (X_test, y_test), callbacks = [earlystop])

In [ ]:
model_loss = pd.DataFrame(model.history.history)

model_loss.plot()

###Tackling Overfitting using Dropout Layer

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
model = Sequential()

model.add(Dense(30, activation = 'relu'))
model.add(Dropout(0.5))                                    ## Common values is 0.2 - 0.5

model.add(Dense(15, activation = 'relu'))
model.add(Dropout(0.5))

#Binary Classification
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 25)

model.fit(x = X_train, y = y_train, epochs = 600, validation_data = (X_test, y_test), callbacks = [earlystop])

In [ ]:
model_loss = pd.DataFrame(model.history.history)

model_loss.plot()

### Evaluation

**From Tensorflow 2.6 predict_classes method has been removed**\n
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
predict_x = model.predict(X_test)
predictions = (predict_x > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
print(classification_report(y_test, predictions))